In [3]:
import os
import cv2
import numpy as np

# Directorios
MASKS_DIR = "../data/masks"  # Carpeta con las máscaras originales
OUTPUT_DIR = "../data/cleaned_masks_erosion"  # Carpeta de salida

os.makedirs(OUTPUT_DIR, exist_ok=True)

def clean_mask_with_erosion(mask_path, kernel_size=3, iterations=1, do_dilation=True):
    """
    1. Carga la máscara (incluyendo canal alfa si existe).
    2. Binariza (umbral).
    3. Aplica erosión con un kernel dado (para separar la mano de objetos pegados).
    4. Detecta componentes conectados y conserva la más grande (la mano).
    5. (Opcional) Vuelve a dilatar para recuperar el contorno original.
    6. Restaura el canal alfa (si existía).
    """

    # Leer la máscara en RGBA si existe (4 canales), o BGR (3 canales)
    mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)

    # Separar canal alfa (o convertir a escala de grises)
    if mask.shape[-1] == 4:
        # RGBA
        alpha_channel = mask[:, :, 3]
    else:
        # Si no hay canal alfa, convertimos a escala de grises
        alpha_channel = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

    # Binarizar [0,255]
    _, binary_mask = cv2.threshold(alpha_channel, 127, 255, cv2.THRESH_BINARY)

    # ---- (1) Erosión para romper la conexión mano-objeto ----
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    eroded_mask = cv2.erode(binary_mask, kernel, iterations=iterations)

    # ---- (2) Conectar componentes y elegir la más grande ----
    num_labels, labels = cv2.connectedComponents(eroded_mask)

    # Hallar el componente más grande (ignorando etiqueta 0 -> fondo)
    max_label = 1
    max_size = 0
    for label_id in range(1, num_labels):
        size = np.sum(labels == label_id)
        if size > max_size:
            max_size = size
            max_label = label_id

    # Crear máscara con solo ese componente
    cleaned_mask = np.zeros_like(eroded_mask)
    cleaned_mask[labels == max_label] = 255

    # ---- (3) (Opcional) Volver a dilatar para recuperar contorno ----
    if do_dilation:
        cleaned_mask = cv2.dilate(cleaned_mask, kernel, iterations=iterations)

    # ---- (4) Reconstruir la imagen de salida con canal alfa (si existía) ----
    if mask.shape[-1] == 4:
        # Creamos una copia RGBA de la imagen original
        output = np.zeros_like(mask)
        # Dejamos intactos los canales RGB originales (opcional) 
        # o ponerlos en negro si solo te importa la alfa
        output[:, :, 0:3] = mask[:, :, 0:3]  
        # Reemplazamos la alfa por cleaned_mask
        output[:, :, 3] = cleaned_mask
        return output
    else:
        # Retornamos una imagen binaria 2D (fondo negro, mano blanca)
        return cleaned_mask

# ---- Procesar todas las máscaras en la carpeta ----
for mask_file in os.listdir(MASKS_DIR):
    if not mask_file.lower().endswith((".png", ".jpg")):
        continue
    
    mask_path = os.path.join(MASKS_DIR, mask_file)
    
    # Ajusta los parámetros según necesites
    # kernel_size = 3, 5, etc.
    # iterations = 1, 2...
    # do_dilation = True/False
    cleaned = clean_mask_with_erosion(
        mask_path, 
        kernel_size=3, 
        iterations=1, 
        do_dilation=True
    )
    
    # Guardar resultado
    output_path = os.path.join(OUTPUT_DIR, mask_file)
    cv2.imwrite(output_path, cleaned)

print("Proceso de limpieza con erosión completado.")


Proceso de limpieza con erosión completado.
